In [25]:
from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torchvision.utils import save_image
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torchvision import datasets, transforms

In [29]:
parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
parser.add_argument('--batch-size', type=int, default=128, metavar='N',
                    help='input batch size for training (default: 128)')
parser.add_argument('--epochs', type=int, default=5, metavar='N',
                    help='number of epochs to train (default: 5)')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='enables CUDA training')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')
parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                    help='how many batches to wait before logging training status')
args = parser.parse_args(args=[])
args.cuda = not args.no_cuda and torch.cuda.is_available()
print(args)

#Sets the seed for generating random numbers. And returns a torch._C.Generator object.
torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)

Namespace(batch_size=128, cuda=False, epochs=5, log_interval=10, no_cuda=False, seed=1)


In [9]:
kwargs = {'num_workers': 1, 'pin_memory': True} if args.cuda else {}
trainset = datasets.MNIST('../data', train=True, download=True,transform=transforms.ToTensor())
train_loader = torch.utils.data.DataLoader(
    trainset,
    batch_size=args.batch_size, shuffle=True, **kwargs)
testset= datasets.MNIST('../data', train=False, transform=transforms.ToTensor())
test_loader = torch.utils.data.DataLoader(
    testset,
    batch_size=args.batch_size, shuffle=True, **kwargs)
image, label = trainset[0]  
print(len(trainset))
print(image.size())
image, label = testset[0]  
print(len(testset))
print(image.size())

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../data\MNIST\raw\train-images-idx3-ubyte.gz to ../data\MNIST\raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../data\MNIST\raw\train-labels-idx1-ubyte.gz to ../data\MNIST\raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../data\MNIST\raw\t10k-images-idx3-ubyte.gz to ../data\MNIST\raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ../data\MNIST\raw

60000
torch.Size([1, 28, 28])
10000
torch.Size([1, 28, 28])


F:\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [10]:
x = torch.randn(4, 4)
y = x.view(16)
z = x.view(-1, 16)

In [34]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(784, 512)
        self.fc21 = nn.Linear(512, 2)
        self.fc22 = nn.Linear(512, 2)
        self.fc3 = nn.Linear(2, 512)
        self.fc4 = nn.Linear(512, 784)

        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def encode(self, x):
        h1 = self.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparametrize(self, mu, logvar):
        std = logvar.mul(0.5).exp_()
        eps = Variable(std.data.new(std.size()).normal_())
        return eps.mul(std).add_(mu)

    def decode(self, z):
        h3 = self.relu(self.fc3(z))
        return self.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 784))
        z = self.reparametrize(mu, logvar)
        return self.decode(z), mu, logvar

model = VAE()
if args.cuda:
    model.cuda()

In [37]:
reconstruction_function = nn.BCELoss()
reconstruction_function.size_average = False

def loss_function(recon_x, x, mu, logvar):
    BCE = reconstruction_function(recon_x, x.view(-1, 784))

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD_element = mu.pow(2).add_(logvar.exp()).mul_(-1).add_(1).add_(logvar)
    KLD = torch.sum(KLD_element).mul_(-0.5)

    return BCE + KLD


optimizer = optim.Adam(model.parameters(), lr=0.0003)

In [22]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = Variable(data)
        if args.cuda:
            data = data.cuda()
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss+=loss.item()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)))
            

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))


def test(epoch):
    model.eval()
    test_loss = 0
    for data, _ in test_loader:
        if args.cuda:
            data = data.cuda()
        data = Variable(data, volatile=True)
        recon_batch, mu, logvar = model(data)
        test_loss += loss_function(recon_batch, data, mu, logvar).item()
    
    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [38]:
for epoch in range(1, args.epochs + 1):
    train(epoch)
    test(epoch)

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.011404
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.006875
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.004924
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.003943
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.003597
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.003251
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.003022
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.002900
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.002871
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.002697
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.002611
Train Epoch: 1 [14080/60000 (23%)]	Loss: 0.002523
Train Epoch: 1 [15360/60000 (26%)]	Loss: 0.002621
Train Epoch: 1 [16640/60000 (28%)]	Loss: 0.002496
Train Epoch: 1 [17920/60000 (30%)]	Loss: 0.002529
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.002448
Train Epoch: 1 [20480/60000 (34%)]	Loss: 0.002450
Train Epoch: 1 [21760/60000 (36%)]	Loss: 0.002569
Train Epoch: 1 [23040/60000 (38%)]	Loss: 0.002350
Train Epoch: 1 [24320/60000 (41%)]	Loss: 0.002318
Train Epoch: 1 [

<ipython-input-22-21b5d308a8bf>:31: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data = Variable(data, volatile=True)


====> Test set loss: 0.0022
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.002160
Train Epoch: 2 [1280/60000 (2%)]	Loss: 0.002091
Train Epoch: 2 [2560/60000 (4%)]	Loss: 0.002211
Train Epoch: 2 [3840/60000 (6%)]	Loss: 0.002090
Train Epoch: 2 [5120/60000 (9%)]	Loss: 0.002172
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.002133
Train Epoch: 2 [7680/60000 (13%)]	Loss: 0.002084
Train Epoch: 2 [8960/60000 (15%)]	Loss: 0.002158
Train Epoch: 2 [10240/60000 (17%)]	Loss: 0.002104
Train Epoch: 2 [11520/60000 (19%)]	Loss: 0.002090
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.002166
Train Epoch: 2 [14080/60000 (23%)]	Loss: 0.002134
Train Epoch: 2 [15360/60000 (26%)]	Loss: 0.002171
Train Epoch: 2 [16640/60000 (28%)]	Loss: 0.002143
Train Epoch: 2 [17920/60000 (30%)]	Loss: 0.002126
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.002145
Train Epoch: 2 [20480/60000 (34%)]	Loss: 0.002154
Train Epoch: 2 [21760/60000 (36%)]	Loss: 0.002050
Train Epoch: 2 [23040/60000 (38%)]	Loss: 0.002173
Train Epoch: 2 [24320/60000 (41%)]	Los

Train Epoch: 5 [25600/60000 (43%)]	Loss: 0.002103
Train Epoch: 5 [26880/60000 (45%)]	Loss: 0.002101
Train Epoch: 5 [28160/60000 (47%)]	Loss: 0.002054
Train Epoch: 5 [29440/60000 (49%)]	Loss: 0.002081
Train Epoch: 5 [30720/60000 (51%)]	Loss: 0.002054
Train Epoch: 5 [32000/60000 (53%)]	Loss: 0.002068
Train Epoch: 5 [33280/60000 (55%)]	Loss: 0.002152
Train Epoch: 5 [34560/60000 (58%)]	Loss: 0.002072
Train Epoch: 5 [35840/60000 (60%)]	Loss: 0.002125
Train Epoch: 5 [37120/60000 (62%)]	Loss: 0.001984
Train Epoch: 5 [38400/60000 (64%)]	Loss: 0.002092
Train Epoch: 5 [39680/60000 (66%)]	Loss: 0.002029
Train Epoch: 5 [40960/60000 (68%)]	Loss: 0.002025
Train Epoch: 5 [42240/60000 (70%)]	Loss: 0.002031
Train Epoch: 5 [43520/60000 (72%)]	Loss: 0.001989
Train Epoch: 5 [44800/60000 (75%)]	Loss: 0.002113
Train Epoch: 5 [46080/60000 (77%)]	Loss: 0.002070
Train Epoch: 5 [47360/60000 (79%)]	Loss: 0.002110
Train Epoch: 5 [48640/60000 (81%)]	Loss: 0.001978
Train Epoch: 5 [49920/60000 (83%)]	Loss: 0.002098


In [39]:
epoch = 0
for data, _ in test_loader:
    data = Variable(data, volatile=True)
    recon_batch, mu, logvar = model(data)
    fake_images = recon_batch.view(-1, 1, 28, 28)
    save_image(fake_images, '..\\data\\photo\\fake_images-{}.png'.format(epoch + 1))
    epoch += 1
    if epoch == 10: break

<ipython-input-39-0a81c7afef98>:3: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data = Variable(data, volatile=True)


In [40]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torchvision.utils import save_image


def loss_function(recon_x, x, mu, logvar):
    """
    :param recon_x: generated image
    :param x: original image
    :param mu: latent mean of z
    :param logvar: latent log variance of z
    """
    BCE_loss = nn.BCELoss(reduction='sum')
    reconstruction_loss = BCE_loss(recon_x, x)
    KL_divergence = -0.5 * torch.sum(1+logvar-torch.exp(logvar)-mu**2)
    #KLD_ele = mu.pow(2).add_(logvar.exp()).mul_(-1).add_(1).add_(logvar)
    #KLD = torch.sum(KLD_ele).mul_(-0.5)
    print(reconstruction_loss, KL_divergence)

    return reconstruction_loss + KL_divergence


class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        self.fc1 = nn.Linear(784, 400)
        self.fc2_mean = nn.Linear(400, 20)
        self.fc2_logvar = nn.Linear(400, 20)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 784)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc2_mean(h1), self.fc2_logvar(h1)

    def reparametrization(self, mu, logvar):
        # sigma = 0.5*exp(log(sigma^2))= 0.5*exp(log(var))
        std = 0.5 * torch.exp(logvar)
        # N(mu, std^2) = N(0, 1) * std + mu
        z = torch.randn(std.size()) * std + mu
        return z

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparametrization(mu, logvar)
        return self.decode(z), mu, logvar


transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5]),
])

trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True)

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False)

vae = VAE()
optimizer = torch.optim.Adam(vae.parameters(), lr=0.0003)

# Training
def train(epoch):
    vae.train()
    all_loss = 0.
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to('cpu'), targets.to('cpu')
        real_imgs = torch.flatten(inputs, start_dim=1)

        # Train Discriminator
        gen_imgs, mu, logvar = vae(real_imgs)
        loss = loss_function(gen_imgs, real_imgs, mu, logvar)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        all_loss += loss.item()
        print('Epoch {}, loss: {:.6f}'.format(epoch, all_loss/(batch_idx+1)))
        # Save generated images for every epoch
    fake_images = gen_imgs.view(-1, 1, 28, 28)
    save_image(fake_images, '..\\data\\photo\\fake_images-{}.png'.format(epoch + 1))



for epoch in range(20):
    train(epoch)

torch.save(vae.state_dict(), '..\\data\\vae.pth')

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

...

tensor(-7221645., grad_fn=<BinaryCrossEntropyBackward>) tensor(6632.1177, grad_fn=<MulBackward0>)
Epoch 19, loss: -7326413.695513
tensor(-7301556.5000, grad_fn=<BinaryCrossEntropyBackward>) tensor(6797.6216, grad_fn=<MulBackward0>)
Epoch 19, loss: -7326212.073248
tensor(-7366994., grad_fn=<BinaryCrossEntropyBackward>) tensor(6483.7461, grad_fn=<MulBackward0>)
Epoch 19, loss: -7326429.151899
tensor(-7379999., grad_fn=<BinaryCrossEntropyBackward>) tensor(6676.9980, grad_fn=<MulBackward0>)
Epoch 19, loss: -7326724.075472
tensor(-7323908., grad_fn=<BinaryCrossEntropyBackward>) tensor(6632.4497, grad_fn=<MulBackward0>)
Epoch 19, loss: -7326665.021875
tensor(-7377231.5000, grad_fn=<BinaryCrossEntropyBackward>) tensor(6713.0449, grad_fn=<MulBackward0>)
Epoch 19, loss: -7326937.403727
tensor(-7231898.5000, grad_fn=<BinaryCrossEntropyBackward>) tensor(6545.4062, grad_fn=<MulBackward0>)
Epoch 19, loss: -7326310.339506
tensor(-7355172., grad_fn=<BinaryCrossEntropyBackward>) tensor(6730.2729, grad

tensor(-7093681., grad_fn=<BinaryCrossEntropyBackward>) tensor(6435.9326, grad_fn=<MulBackward0>)
Epoch 19, loss: -7332329.827273
tensor(-7204443.5000, grad_fn=<BinaryCrossEntropyBackward>) tensor(6478.5469, grad_fn=<MulBackward0>)
Epoch 19, loss: -7331721.841629
tensor(-7408071.5000, grad_fn=<BinaryCrossEntropyBackward>) tensor(6745.0830, grad_fn=<MulBackward0>)
Epoch 19, loss: -7332035.376126
tensor(-7391513., grad_fn=<BinaryCrossEntropyBackward>) tensor(6704.0044, grad_fn=<MulBackward0>)
Epoch 19, loss: -7332272.029148
tensor(-7397131.5000, grad_fn=<BinaryCrossEntropyBackward>) tensor(6525.6997, grad_fn=<MulBackward0>)
Epoch 19, loss: -7332532.448661
tensor(-7383886., grad_fn=<BinaryCrossEntropyBackward>) tensor(6733.1260, grad_fn=<MulBackward0>)
Epoch 19, loss: -7332730.762222
tensor(-7391586., grad_fn=<BinaryCrossEntropyBackward>) tensor(6766.8643, grad_fn=<MulBackward0>)
Epoch 19, loss: -7332961.241150
tensor(-7336376., grad_fn=<BinaryCrossEntropyBackward>) tensor(6694.5869, grad

tensor(-7385139., grad_fn=<BinaryCrossEntropyBackward>) tensor(6846.8125, grad_fn=<MulBackward0>)
Epoch 19, loss: -7329627.433566
tensor(-7274399., grad_fn=<BinaryCrossEntropyBackward>) tensor(6657.2339, grad_fn=<MulBackward0>)
Epoch 19, loss: -7329411.804878
tensor(-7365567., grad_fn=<BinaryCrossEntropyBackward>) tensor(6761.8403, grad_fn=<MulBackward0>)
Epoch 19, loss: -7329513.864583
tensor(-7393537.5000, grad_fn=<BinaryCrossEntropyBackward>) tensor(6739.5049, grad_fn=<MulBackward0>)
Epoch 19, loss: -7329712.079585
tensor(-7424676., grad_fn=<BinaryCrossEntropyBackward>) tensor(6827.2461, grad_fn=<MulBackward0>)
Epoch 19, loss: -7330016.000000
tensor(-7550623.5000, grad_fn=<BinaryCrossEntropyBackward>) tensor(6661.5479, grad_fn=<MulBackward0>)
Epoch 19, loss: -7330751.209622
tensor(-7673095., grad_fn=<BinaryCrossEntropyBackward>) tensor(6636.1836, grad_fn=<MulBackward0>)
Epoch 19, loss: -7331900.893836
tensor(-7417181., grad_fn=<BinaryCrossEntropyBackward>) tensor(6684.3701, grad_fn=

tensor(-7411927., grad_fn=<BinaryCrossEntropyBackward>) tensor(6702.7407, grad_fn=<MulBackward0>)
Epoch 19, loss: -7339913.517192
tensor(-7367888.5000, grad_fn=<BinaryCrossEntropyBackward>) tensor(6717.1226, grad_fn=<MulBackward0>)
Epoch 19, loss: -7339974.254286
tensor(-7510529., grad_fn=<BinaryCrossEntropyBackward>) tensor(6676.5229, grad_fn=<MulBackward0>)
Epoch 19, loss: -7340441.143875
tensor(-7329073., grad_fn=<BinaryCrossEntropyBackward>) tensor(6472.6597, grad_fn=<MulBackward0>)
Epoch 19, loss: -7340390.460227
tensor(-7433075., grad_fn=<BinaryCrossEntropyBackward>) tensor(6504.4932, grad_fn=<MulBackward0>)
Epoch 19, loss: -7340634.596317
tensor(-7194002., grad_fn=<BinaryCrossEntropyBackward>) tensor(6650.0786, grad_fn=<MulBackward0>)
Epoch 19, loss: -7340201.594633
tensor(-7513095.5000, grad_fn=<BinaryCrossEntropyBackward>) tensor(6538.6895, grad_fn=<MulBackward0>)
Epoch 19, loss: -7340670.201408
tensor(-7397320.5000, grad_fn=<BinaryCrossEntropyBackward>) tensor(6667.0205, grad

tensor(-7303174., grad_fn=<BinaryCrossEntropyBackward>) tensor(6872.4785, grad_fn=<MulBackward0>)
Epoch 19, loss: -7341549.787621
tensor(-7239470., grad_fn=<BinaryCrossEntropyBackward>) tensor(6587.6592, grad_fn=<MulBackward0>)
Epoch 19, loss: -7341286.670702
tensor(-7323448., grad_fn=<BinaryCrossEntropyBackward>) tensor(6699.7324, grad_fn=<MulBackward0>)
Epoch 19, loss: -7341227.399758
tensor(-7221022., grad_fn=<BinaryCrossEntropyBackward>) tensor(6718.4814, grad_fn=<MulBackward0>)
Epoch 19, loss: -7340921.559036
tensor(-7321164., grad_fn=<BinaryCrossEntropyBackward>) tensor(6536.1519, grad_fn=<MulBackward0>)
Epoch 19, loss: -7340858.353365
tensor(-7567550., grad_fn=<BinaryCrossEntropyBackward>) tensor(6560.0049, grad_fn=<MulBackward0>)
Epoch 19, loss: -7341386.247002
tensor(-7508311., grad_fn=<BinaryCrossEntropyBackward>) tensor(6867.2080, grad_fn=<MulBackward0>)
Epoch 19, loss: -7341769.160287
tensor(-7195868., grad_fn=<BinaryCrossEntropyBackward>) tensor(6744.5410, grad_fn=<MulBack

In [41]:
epoch = 0
for batch_idx, (inputs, targets) in enumerate(testloader):
        inputs, targets = inputs.to('cpu'), targets.to('cpu')
        real_imgs = torch.flatten(inputs, start_dim=1)

        # Train Discriminator
        gen_imgs, mu, logvar = vae(real_imgs)
        fake_images = gen_imgs.view(-1, 1, 28, 28)
        save_image(fake_images, '..\\data\\photo\\fake_images-{}.png'.format(epoch + 1))
        epoch += 1
        if epoch == 11:break